In [4]:
import os
import shutil
from enum import Enum
import json
from csnake import CodeWriter, Struct, Variable, Function, FormattedLiteral
import numpy as np

In [5]:
data = None
with open("src/data.json","r") as f:
    data = json.load(f)

data

{'sensors': [{'name': 'polder11/gate13/01',
   'type': '1',
   'port_pin1': [1, 9],
   'port_pin2': [0, 8]},
  {'name': 'polder11/gate13/02',
   'type': '1',
   'port_pin1': [0, 6],
   'port_pin2': [0, 27]}]}

## Was muss generiter werden:
- NUM_SENSORS
- Liste/Array der Typen
- Init code für die alarm_cb_args


In [ ]:
class SensorTypeID(Enum):
    DWAX = "SENSOR_TYPE_ID_DWAX509M183X0" 
    REED_NC = "SENSOR_TYPE_ID_REED_SWITCH_NC" 
    REED_NO = "SENSOR_TYPE_ID_REED_SWITCH_NO"


class CodeBuffer:

    def __init__(self):
        self.lines = []
        self.comment_active = False

    def add_line(self,line,comment,ignore_indent=False):
        if comment:
            line = f'/* {comment} */'    
        if ignore_indent:
            line += "\n"
        if self.comment_active:
            line += f'* {line}'
        self.lines.append(line)

    def add_lines(self,lines):
        self.lines.extend(lines)
    
    def add_new_line(self):
        self.lines.append('\n')
        
    def include(self,inc):
        self.add_line(f'#include "{inc}"')

    def add_define(self,name, val):
        self.add_line(f'#define "{name} {val}"')
    
    def add_macro(self,name, val):
        self.add_line(f'#define "{name}{val}"')

    def start_comment(self):
        self.comment_active = True
        self.add_line('/*',ignore_indent=True)
    
    def end_comment(self):
        self.comment_active = False
        self.add_new_line()
        self.add_line('*/')


In [ ]:

cw = CodeWriter()
cw.add_lines([
    "#ifndef SENSOR_CONFIG_H_",
    "#define SENSOR_CONFIG_H_",
    "\n"
])
cw.include("sensors.h")
cw.include("dwax509m183x0.h")
cw.include("reed_sensor_driver.h")
cw.add_line(ignore_indent=True)




cw.add_define("SENSOR_TYPE_ID_DWAX509M183X0",1)
cw.add_define("SENSOR_TYPE_ID_REED_SWITCH_NC",2)
cw.add_define("SENSOR_TYPE_ID_REED_SWITCH_NO",3)
cw.add_line(ignore_indent=True)


cw.add_define("REED_SENSOR_DEBOUNCE_MS",60)
cw.add_line(ignore_indent=True)

cw.start_comment()
cw.add_lines([
            "Macro to encode sensor type and global sensor id in a 16-Bit integer.",
            "---------------------------------",
            "|   Sensor Type (8 bits)     |   Sensor Number (8 bits)   |",
            "|----------------------------|----------------------------|",
            "|  7  6  5  4  3  2  1  0    |  7  6  5  4  3  2  1  0    |",
            "|---------------------------------------------------------|",
            ])
cw.end_comment()
cw.add_line("#define ENCODE_SENSOR_TYPE_ID(type, id) ((type) << 8 | (id))")
cw.add_line(ignore_indent=True)


# add sensor base type
cw.add_lines([
    "typedef union {",
    "\treed_sensor_driver_t reed_sensor;",
    "\tdwax509m183x0_t inductive_sensor;",
    "} sensor_base_type_t;",
    "\n"
])

cw.add_lines([
    "typedef union {",
    "\treed_sensor_driver_params_t reed_sensor_params;",
    "\tdwax509m183x0_params_t inductive_sensor_params;",
    "} sensor_base_params_t;",
    "\n"
])


cw.add_line(comment='-' * 30 + 'Sensor Declaration' + '-' * 30)
cw.add_line(ignore_indent=True)

num_sensors = len(data["sensors"]) 
cw.add_line(comment='Number of physical sensors connected (some sensors have multiple contacts e.g. reed sensors)')
cw.add_define("NUM_SENSORS",num_sensors)
cw.add_line(ignore_indent=True)

num_unique_sensor_values = num_sensors + sum([1 for s in data["sensors"] if int(s["type"]) == 1 ])
cw.add_define("NUM_UNIQUE_SENSOR_VALUES",num_unique_sensor_values)


cw.add_line("static alarm_cb_args_t alarm_cb_args[NUM_UNIQUE_SENSOR_VALUES];")
cw.add_line(ignore_indent=True)




sensors = []
init_code_lines = []
id_counter = 0
for i,s in enumerate(data["sensors"]):
    s_type = None 

    name_base= ""

    if int(s["type"]) == 1: # REED
        name_base = "reed"
        s_type = "reed_sensor_driver_t"


    elif int(s["type"]) == 2: # INDUCTIVE
        name_base = "dwax"
        s_type = "dwax509m183x0_t"


    else:
        TypeError(f"Type {s['type']} not supported!")        


    ## create sensor handle variable

    s_name = f"sensor_{i+1}_{name_base}"
    # s_var = Variable(
    #     s_name,
    #     primitive=f"{s_type}",
    # )


    # cw.add_define(f"SENSOR_{i+1}",i)

    
    ## create sensor contact defines and callback arguments
    if int(s["type"]) == 1:
        s_id_nc = f"{s_name.upper()}_NC_ID"
        cw.add_define(s_id_nc,id_counter)
        
        init_code_lines.extend([
        f'''
alarm_cb_args[{s_id_nc}].pid = thread_getpid();
alarm_cb_args[{s_id_nc}].msg.type = ENCODE_SENSOR_TYPE_ID({SensorTypeID.REED_NC.value},{id_counter});
alarm_cb_args[{s_id_nc}].msg.content.ptr = (void *)&registered_sensors[{i}]; 

         '''
        ])

        id_counter += 1

        ## no 
        s_id_no = f"{s_name.upper()}_NO_ID"

        cw.add_define(s_id_no,id_counter)
        
        init_code_lines.extend([
        f'''
alarm_cb_args[{s_id_no}].pid = thread_getpid();
alarm_cb_args[{s_id_no}].msg.type = ENCODE_SENSOR_TYPE_ID({SensorTypeID.REED_NO.value},{id_counter});
alarm_cb_args[{s_id_no}].msg.content.ptr = (void *)&registered_sensors[{i}];

         '''
        ])

        id_counter += 1

        ## driver init
        init_code_lines.extend([
        f'''
// first cast to specific param type and then to base params type for the array.
registered_sensors_params[{i}] = (sensor_base_params_t) (reed_sensor_driver_params_t){{
                                        .nc_pin = GPIO_PIN({s["port_pin1"][0]},{s["port_pin1"][1]}),
                                        .no_pin = GPIO_PIN({s["port_pin2"][0]},{s["port_pin2"][1]}),
                                        .nc_int_flank = GPIO_BOTH,
                                        .no_int_flank = GPIO_BOTH,
                                        .nc_callback = reed_nc_callback,
                                        .no_callback = reed_no_callback,
                                        .nc_callback_args = (void *)&alarm_cb_args[{s_id_nc}],
                                        .no_callback_args = (void *)&alarm_cb_args[{s_id_no}],
                                        .use_external_pulldown = false,
                                        .debounce_ms = REED_SENSOR_DEBOUNCE_MS}};
reed_sensor_driver_init(&registered_sensors[{i}].reed_sensor, &registered_sensors_params[{i}].reed_sensor_params);
        '''
        ])




    elif int(s["type"]) == 2:
        s_id = f"{s_name.upper()}_ID"
        cw.add_define(s_id,id_counter)
        init_code_lines.extend([
        f'''
alarm_cb_args[{s_id}].pid = thread_getpid();
alarm_cb_args[{s_id}].msg.type = ENCODE_SENSOR_TYPE_ID({SensorTypeID.DWAX.value},{id_counter});
alarm_cb_args[{s_id}].msg.content.ptr = (void *)&registered_sensors[{i}];

         '''
        ])
        id_counter += 1

    ## add sensor handle variable declaration
    # cw.add_variable_declaration(s_var)
    cw.add_line(ignore_indent=True)
    # sensors.append(s_var)



cw.add_line(ignore_indent=True)


registered_sensors = Variable(
    "registered_sensors",
    primitive="sensor_base_type_t",
    array=num_sensors
)

registered_sensors_params = Variable(
    "registered_sensors_params",
    primitive="sensor_base_params_t",
    array=num_sensors
)



cw.add_variable_declaration(registered_sensors)
cw.add_variable_declaration(registered_sensors_params)



init_func = Function(
    "init_sensors",
    return_type="int",
)
init_func.add_code(init_code_lines + ["return 0;"])

#todo: ADD driver initialization. example:
#  reed_sensor_driver_params_t params = {.nc_pin = nc_pin,
#                                           .no_pin = no_pin,
#                                           .nc_int_flank = GPIO_BOTH,
#                                           .no_int_flank = GPIO_BOTH,
#                                           //   .nc_callback = reed_nc_callback_and_dwax_trigger,
#                                           .nc_callback = reed_nc_callback,
#                                           .no_callback = reed_no_callback,
#                                         //   .nc_callback_args = (void *)&alarm_cb_args, // Note: passing the whole array to be able to access the callback of dawx too
#                                           .nc_callback_args = (void *)&alarm_cb_args[1],
#                                           .no_callback_args = (void *)&alarm_cb_args[2],
#                                           .use_external_pulldown = false,
#                                           .debounce_ms = REED_SENSOR_DEBOUNCE_MS};
#     reed_sensor_driver_init(&sensor_02, &params);


cw.add_function_definition(init_func)


cw.add_line("#endif // SENSOR_CONFIG_H_")
print(cw)

with open("src/sensor_config.h","w") as f:
    f.writelines(cw.code)


shutil.copyfile("src/sensor_config.h","/home/timon/docs/uni/projects/resuce-mate-sensors/silenos/sensor_config.h")

#ifndef SENSOR_CONFIG_H_
#define SENSOR_CONFIG_H_

#include "sensors.h"
#include "dwax509m183x0.h"
#include "reed_sensor_driver.h"

#define SENSOR_TYPE_ID_DWAX509M183X0 1
#define SENSOR_TYPE_ID_REED_SWITCH_NC 2
#define SENSOR_TYPE_ID_REED_SWITCH_NO 3

#define REED_SENSOR_DEBOUNCE_MS 60

/*
* Macro to encode sensor type and global sensor id in a 16-Bit integer.
* ---------------------------------
* |   Sensor Type (8 bits)     |   Sensor Number (8 bits)   |
* |----------------------------|----------------------------|
* |  7  6  5  4  3  2  1  0    |  7  6  5  4  3  2  1  0    |
* |---------------------------------------------------------|
*/
#define ENCODE_SENSOR_TYPE_ID(type, id) ((type) << 8 | (id))

typedef union {
	reed_sensor_driver_t reed_sensor;
	dwax509m183x0_t inductive_sensor;
} sensor_base_type_t;

typedef union {
	reed_sensor_driver_params_t reed_sensor_params;
	dwax509m183x0_params_t inductive_sensor_params;
} sensor_base_params_t;

/* ------------------------------Sensor 

'/home/timon/docs/uni/projects/resuce-mate-sensors/silenos/sensor_config.h'

In [8]:
print("DWAX:   {:>08b}".format((SensorTypeID.DWAX.value << 8) | (0)))
print("REED NC {:>08b}".format((SensorTypeID.REED_NC.value << 8) | (1)))
print("REED NO {:>08b}".format((SensorTypeID.REED_NO.value << 8) | (2)))


print("DWAX:   {:>08b}".format((SensorTypeID.DWAX.value << 1) | (0)))
print("REED NC {:>08b}".format((SensorTypeID.REED_NC.value << 1) | (1)))
print("REED NO {:>08b}".format((SensorTypeID.REED_NO.value << 1) | (2)))


TypeError: unsupported operand type(s) for <<: 'str' and 'int'

In [ ]:
f'''
reed_sensor_driver_params_t params = {{.nc_pin = {23},
                                          .no_pin = no_pin,
                                          .nc_int_flank = GPIO_BOTH,
                                          .no_int_flank = GPIO_BOTH,
                                          //   .nc_callback = reed_nc_callback_and_dwax_trigger,
                                          .nc_callback = reed_nc_callback,
                                          .no_callback = reed_no_callback,
                                        //   .nc_callback_args = (void *)&alarm_cb_args, // Note: passing the whole array to be able to access the callback of dawx too
                                          .nc_callback_args = (void *)&alarm_cb_args[1],
                                          .no_callback_args = (void *)&alarm_cb_args[2],
                                          .use_external_pulldown = false,
                                          .debounce_ms = REED_SENSOR_DEBOUNCE_MS}};
'''


'\nreed_sensor_driver_params_t params = {.nc_pin = 23,\n                                          .no_pin = no_pin,\n                                          .nc_int_flank = GPIO_BOTH,\n                                          .no_int_flank = GPIO_BOTH,\n                                          //   .nc_callback = reed_nc_callback_and_dwax_trigger,\n                                          .nc_callback = reed_nc_callback,\n                                          .no_callback = reed_no_callback,\n                                        //   .nc_callback_args = (void *)&alarm_cb_args, // Note: passing the whole array to be able to access the callback of dawx too\n                                          .nc_callback_args = (void *)&alarm_cb_args[1],\n                                          .no_callback_args = (void *)&alarm_cb_args[2],\n                                          .use_external_pulldown = false,\n                                          .debounce_ms = REED_SENSOR_